# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Условия задачи:**

* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

In [1]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

pd.options.mode.chained_assignment = None

import warnings
warnings.filterwarnings('ignore')

Загрузка данных:

In [2]:
#geo_data_0 = pd.read_csv('geo_data_0.csv')
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')

In [3]:
#geo_data_1 = pd.read_csv('geo_data_1.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')

In [4]:
#geo_data_2 = pd.read_csv('geo_data_2.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [5]:
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)

# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)

# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None) 

Описание данных:
* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

Изучим данные:

In [6]:
# 1-я таблица
display(geo_data_0.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [7]:
# 1-я таблица
display(geo_data_0.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [8]:
# 1-я таблица
display(geo_data_0.describe())

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [9]:
# 2-я таблица
display(geo_data_1.info())
display(geo_data_1.head())
display(geo_data_1.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [10]:
# 3-я таблица
display(geo_data_2.info())
display(geo_data_2.head())
display(geo_data_2.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Проверим, есть ли дубликаты в таблицах:

In [11]:
# 1-я таблица
display(geo_data_0.duplicated().sum())

0

In [12]:
# 2-я таблица
display(geo_data_1.duplicated().sum())

0

In [13]:
# 3-я таблица
display(geo_data_2.duplicated().sum())

0

### Вывод

Были загружены и проанализированы данные:

* Данные синтетические (из условий задачи)
* Пропусков нет
* Признаки f0, f1, f2
* Целевой признак product
* Есть ненужный столбец id, который будет удален при разбиении данных на тренировочную и валидационную выборки

## Обучение и проверка модели

Напишем функцию, которая примет датафрейм, разобъет его на обучающую и валидационную выборки в соотношении 75:25, обучит модель Линейной регрессии, сохранит предсказания и правильные ответы на валидационной выборке и напечатает на экране средний запас предсказанного сырья и RMSE модели:

In [14]:
def get_answer(df):
    # разбиение датафрейма на обучающую и валидационную выборки в соотношении 75:25
    features = df.drop(['id', 'product'], axis = 1)
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
   
    # обучение модели Линейной регрессии, сохранение предсказания и правильных ответов на валидационной выборке
    model = LinearRegression(normalize=True, n_jobs=-1)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    
    # вывод на экране средннго запаса предсказанного сырья и RMSE модели
    mse = mean_squared_error(target_valid, predictions)
    print('Средний запас сырья, тыс. баррелей:', predictions.mean())
    print('RMSE модели:', mse ** 0.5)
    
    return predictions, target_valid, mse

In [15]:
print('1-й регион')
predictions_1, target_valid_1, mse_1 = get_answer(geo_data_0)

1-й регион
Средний запас сырья, тыс. баррелей: 92.59256778438035
RMSE модели: 37.5794217150813


In [16]:
print('2-й регион')
predictions_2, target_valid_2, mse_2 = get_answer(geo_data_1)

2-й регион
Средний запас сырья, тыс. баррелей: 68.728546895446
RMSE модели: 0.8930992867756157


In [17]:
print('3-й регион')
predictions_3, target_valid_3, mse_3 = get_answer(geo_data_2)

3-й регион
Средний запас сырья, тыс. баррелей: 94.96504596800489
RMSE модели: 40.02970873393434


Посмотрим на полученные предсказанные значения:

In [18]:
data_1 = {'real_product' : target_valid_1, 'predicted_product' : predictions_1}
data_frame_1 = pd.DataFrame(data_1, columns = ['real_product', 'predicted_product'])

In [19]:
data_frame_1.head(10)

,real_product,predicted_product
71751,10.038645,95.894952
80493,114.551489,77.572583
2655,132.603635,77.892640
53233,169.072125,90.175134
91141,122.325180,70.510088
9539,37.735331,69.127076
8482,139.227191,125.106759
30284,72.491987,87.643849
66393,11.465993,86.035871
99584,110.562031,98.655311


In [20]:
data_2 = {'real_product' : target_valid_2, 'predicted_product' : predictions_2}
data_frame_2 = pd.DataFrame(data_2, columns = ['real_product', 'predicted_product'])
data_frame_2.head(10)

,real_product,predicted_product
71751,80.859783,82.663314
80493,53.906522,54.431786
2655,30.132364,29.748760
53233,53.906522,53.552133
91141,0.000000,1.243856
9539,110.992147,111.438490
8482,137.945408,137.134374
30284,84.038886,82.888902
66393,110.992147,110.897311
99584,26.953261,29.219306


In [21]:
data_3 = {'real_product' : target_valid_3, 'predicted_product' : predictions_3}
data_frame_3 = pd.DataFrame(data_3, columns = ['real_product', 'predicted_product'])
data_frame_3.head(10)

,real_product,predicted_product
71751,61.212375,93.599633
80493,41.850118,75.105159
2655,57.776581,90.066809
53233,100.053761,105.162375
91141,109.897122,115.303310
9539,105.904629,121.939197
8482,51.646180,119.053040
30284,173.162121,75.396575
66393,113.143072,111.400543
99584,56.803215,84.029320


### Вывод

Полученные средние запасы сырья по регионам:
1. 92.59 тыс. баррелей
2. 68.73 тыс. баррелей
3. 94.97 тыс. баррелей

В 1 и 3 регионах запас сырья выше, но RMSE модели ниже во 2 регионе (0,89, возможно модель переобучилась).

Полученные RMSE в 1 регионе - 37.58, в 3 - 40.03.

## Подготовка к расчёту прибыли

In [22]:
# бюджет на разработку скважин в регионе
budget = 10_000_000_000 

# сколько точек исследуют при разведке региона
chosen_points = 500

# сколько лучших точек выбирают
best_points = 200

# один баррель сырья приносит 450 рублей дохода
price_per_thousand_barrel = 450*1000

# нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%
maximum_loss_probability = 0.025

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины:

In [23]:
profit_per_point = budget/best_points
sufficient_volume = profit_per_point/price_per_thousand_barrel

print(f'Сколько точка должна принести прибыли: {profit_per_point} миллионов рублей')
print(f'Достаточный объём сырья для безубыточной разработки новой скважины: {sufficient_volume:.3f} тыс. баррелей')
print(f'Достаточный обънм сырья на весь регион: {budget/price_per_thousand_barrel:.3f} тыс. баррелей')

Сколько точка должна принести прибыли: 50000000.0 миллионов рублей
Достаточный объём сырья для безубыточной разработки новой скважины: 111.111 тыс. баррелей
Достаточный обънм сырья на весь регион: 22222.222 тыс. баррелей


Средний запас сырья в каждом регионе:

In [24]:
print(f'Средний запас сырья на 1 точку в регионе 1: {geo_data_0["product"].mean():.3f} тыс. баррелей')
print(f'Средний запас сырья на 1 точку в регионе 2:  {geo_data_1["product"].mean():.3f} тыс. баррелей')
print(f'Средний запас сырья на 1 точку в регионе 3: {geo_data_2["product"].mean():.3f} тыс. баррелей')

Средний запас сырья на 1 точку в регионе 1: 92.500 тыс. баррелей
Средний запас сырья на 1 точку в регионе 2:  68.825 тыс. баррелей
Средний запас сырья на 1 точку в регионе 3: 95.000 тыс. баррелей


Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

In [25]:
def points_profit(target, predictions):
   # Выберем скважины с максимальными значениями предсказаний.
    predictions_sorted = predictions.sort_values(ascending=False)
    selected_points = target[predictions_sorted.index][:best_points]
    
   # Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
    selected_points_sum = selected_points.sum()
    
   # Рассчитаем прибыль для полученного объёма сырья.
    profit = price_per_thousand_barrel * selected_points_sum - budget
    
    return profit

### Вывод

Необходимый объём сырья для безубыточной разработки новой скважины 111,2 тыс. баррелей, но во всех 3 регионах средний запас меньше (в 1 регионе 92,5 тысячи баррелей, во 2 регионе 68,83 тысячи баррелей, в 3 регионе 95 тысяч баррелей).

Так как исследовать будем 500 скважин, а выбрать 200 самых прибильных, можно найти регион, который подойдет под наши требования, для этого (расчета прибыли) была написана функция.

## Расчёт прибыли и рисков 

In [26]:
print(f'Расчёт прибыли по выбранным скважинам и предсказаниям модели для 1 региона: \
{(points_profit(data_frame_1["real_product"], data_frame_1["predicted_product"]))/1_000_000_000:.6f} млрд.рублей'
     )

Расчёт прибыли по выбранным скважинам и предсказаниям модели для 1 региона: 3.320826 млрд.рублей


In [27]:
print(f'Расчёт прибыли по выбранным скважинам и предсказаниям модели для 2 региона: \
{(points_profit(data_frame_2["real_product"], data_frame_2["predicted_product"]))/1_000_000_000:.6f} млрд.рублей'
     )

Расчёт прибыли по выбранным скважинам и предсказаниям модели для 2 региона: 2.415087 млрд.рублей


In [28]:
print(f'Расчёт прибыли по выбранным скважинам и предсказаниям модели для 3 региона: \
{(points_profit(data_frame_3["real_product"], data_frame_3["predicted_product"]))/1_000_000_000:.6f} млрд.рублей'
     )

Расчёт прибыли по выбранным скважинам и предсказаниям модели для 3 региона: 2.710350 млрд.рублей


Посчитаем риски и прибыль для каждого региона:

In [29]:
state = np.random.RandomState(12345)

In [30]:
def bootstrap_profit(target, predictions):
    #  Техника Bootstrap с 1000 выборок, чтобы найти распределение прибыли
    values = []
    million = 1_000_000
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        pred_subsample = predictions[target_subsample.index]
        profit = points_profit(target, pred_subsample)
        values.append(profit) 
    
    values = pd.Series(values)
    
    # Средняя прибыль, 95%-й доверительный интервал и риск убытков (убыток — это отрицательная прибыль)
    average = values.mean()
    lower = values.quantile(maximum_loss_probability)
    upper = values.quantile(1-maximum_loss_probability)

    print(f'Средняя прибыль: {average/million:.3f} млн.рублей,\n\
95% доверительный интервал от: {lower/million:.3f} млн.рублей до {upper/million:.3f} млн.рублей,\n\
Риски убытка: {((values < 0).mean())*100}%')

In [132]:
print('Данные по 1 региону')
bootstrap_profit(data_frame_1["real_product"], data_frame_1["predicted_product"])

Данные по 1 региону
Средняя прибыль: 392.968 млн.рублей,
95% доверительный интервал от: -110.326 млн.рублей до 887.086 млн.рублей,
Риски убытка: 5.800000000000001%


In [144]:
print('Данные по 2 региону')
bootstrap_profit(data_frame_2["real_product"], data_frame_2["predicted_product"])

Данные по 2 региону
Средняя прибыль: 452.137 млн.рублей,
95% доверительный интервал от: 45.258 млн.рублей до 843.749 млн.рублей,
Риски убытка: 1.0%


In [148]:
print('Данные по 3 региону')
bootstrap_profit(data_frame_3["real_product"], data_frame_3["predicted_product"])

Данные по 3 региону
Средняя прибыль: 395.131 млн.рублей,
95% доверительный интервал от: -124.582 млн.рублей до 906.378 млн.рублей,
Риски убытка: 6.5%


### Вывод

Были посчитаны риски и прибыль для каждого региона:

1 регион:
* Средняя прибыль 392.968 млн.рублей
* Риски убытка 5.8%

2 регион:
* Средняя прибыль 452.137 млн.рублей
* Риски убытка 1.0%

3 регион:
* Средняя прибыль 395.131 млн.рублей
* Риски убытка 6.5%

## Вывод

Исходя из построенной модели, полученных средних данных по запасам сырья и расчетов прибыли и рисков, можно сделать следующие выводы:

* Средний запас во 2 регионе ниже, но модель обученная предсказала запасы лучше, чем в 1 и 3 регионах.
* Риск убытка во 2 регионе самый низкий (1%)
* Во втором регионе с вероятностью 95% не будет убытка.

Поэтому, второй регион можно предложить для разработки скважин.